Amazon NL Apple

In [20]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, StaleElementReferenceException
import time
from bs4 import BeautifulSoup
import pandas as pd

urls = []
chrome_options = Options()
chrome_prefs = {
    "profile.default_content_settings.cookies": 2,
}
chrome_options.add_experimental_option("prefs", chrome_prefs)

# Launch the browser with the custom options
driver = webdriver.Chrome(options=chrome_options)
mainurl = 'https://www.amazon.nl/stores/page/2E412BAF-98AA-41BA-B3A2-765722ADC8FF?ingress=2&visitId=b7466fa9-7305-43b4-a4ea-9679c0108315&ref_=ast_bln&language=en_GB'
driver.get(mainurl)

try:
    # Handle cookies popup
    accept_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.ID, "sp-cc-accept"))
    )
    accept_button.click()
except TimeoutException:
    print("Cookie accept button not found, proceeding without accepting cookies.")

while True:
    try:
        # Wait for the page to load completely and the loading mask to disappear
        WebDriverWait(driver, 20).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, 'loading-mask'))
        )
        
        # Locate and click the "Show More" button, handling stale elements
        show_more_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ShowMoreButton")]'))
        )
        try:
            show_more_button.click()  # Click the "Show More" button
        except StaleElementReferenceException:
            # Re-locate the button in case of a stale element
            show_more_button = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ShowMoreButton")]'))
            )
            show_more_button.click()  # Try clicking again

        time.sleep(5)  # Reduce the sleep duration for testing purposes
        
        # Wait for the product grid to load
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'a.Overlay__overlay__LloCU.ProductGridItem__overlay__IQ3Kw'))
        )
    except TimeoutException:
        print("End of content or timeout reached.")
        break
    except ElementClickInterceptedException:
        print("Element click intercepted, retrying...")
        time.sleep(2)  # Wait for the obstruction to clear
        continue
    except Exception as e:
        print(f"Unexpected error: {e}")
        break

# Parse the page content after clicking 'Show More'
soup = BeautifulSoup(driver.page_source, 'html.parser')
url_tags = soup.find_all('a', class_='Overlay__overlay__LloCU')

for url_tag in url_tags:
    urls.append(f"https://www.amazon.nl{url_tag['href']}&language=en_GB")

driver.quit()
print(f'{len(urls)} URLs Retrieved')

End of content or timeout reached.
96 URLs Retrieved


In [28]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('div', id='cm-cr-dp-review-list')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_pattern2 = re.compile(r'(\d+MP)\s+camera')
        camera_pattern3 = re.compile(r'(\d+ MP)\s+camera')
        camera_pattern4 = re.compile(r'(\d+ MP)\s+MAIN CAMERA')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            match2 = camera_pattern2.search(li.text)
            match3 = camera_pattern3.search(li.text)
            match4 = camera_pattern4.search(li.text)
            if match:
                camera_info = match.group(1)
                break
            if match2:
                camera_info = match2.group(1)
                break
            if match3:
                camera_info = match3.group(1)
                break
            if match4:
                camera_info = match4.group(1)
                break
        camera_features = None  # Placeholder for camera information
        
        # Find the table with camera features
        camera_features_row = soup.find('table', id='productDetails_techSpec_section_1')
        if camera_features_row:
            # Find the specific row containing "Other camera features"
            camera_features_row2 = camera_features_row.find('th', string='Other camera features')
            
            # Ensure the row exists and extract the camera details
            if camera_features_row2:
                camera_features = camera_features_row2.find_next('td').get_text(strip=True)
                
                # Extract the main camera's MP (assuming 'XXMP main camera' format)
                if 'main camera' in camera_features:
                    # Get the part before "main camera" and extract the megapixels
                    camera_info = camera_features.split('main camera')[0].strip().split()[-1]
        
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        color = None  # Initialize color
        
        if color_row:
            # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        
        elif soup.find('table', id ='productDetails_techSpec_section_1'):
            color_row2 = soup.find('th', string='Color')
            if color_row2:
                color = color_row2.find_next('td').get_text(strip=True)
        
        else:
            # Check for color in div with id='variation_color_name'
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = get_text_or_none(selection_row)
        price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                         if soup.find('div', class_='a-section a-spacing-micro') else None)
        if not price:
            price = get_text_or_none(soup.find('span', class_='a-price-whole'))



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': price,
            'Price Before Promotion' : get_text_or_none(soup.find('span', class_='a-price a-text-price').find('span', class_='a-offscreen') if soup.find('span', class_='a-price a-text-price') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('€')
            Website.append('Amazon NL')

# Create a DataFrame to store the scraped data
df_apple = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_apple

Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found


,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Model,...,Camera,Color,Price,Price Before Promotion,Currancy,Rate,URL,IMG,Reviews,Website
0,Apple iPhone 15 Pro Max (1 TB) - Natural titanium,Apple,ios,8 GB,None,1 TB,6.7 Inches,None,None,iPhone 15 Pro Max,...,None,Natural titanium,"€1,695.00","€1,979.00",€,4.6 out of 5 stars,https://www.amazon.nl/Apple-iPhone-15-Pro-Max/...,https://m.media-amazon.com/images/I/81izs2I2Uv...,"[{'reviewer_name': 'Hamid', 'review_rating': '...",Amazon NL
1,Apple iPhone 15 Pro Max (512GB) - Natural tita...,Apple,ios,8 GB,None,512 GB,6.7 Inches,None,None,iPhone 15 Pro Max,...,None,Natural titanium,"€1,461.00","€1,729.00",€,4.6 out of 5 stars,https://www.amazon.nl/Apple-iPhone-Pro-Max-512...,https://m.media-amazon.com/images/I/81izs2I2Uv...,"[{'reviewer_name': 'Hamid', 'review_rating': '...",Amazon NL
2,Apple iPhone 15 Pro Max (256GB) - Natural tita...,Apple,ios,8 GB,None,256 GB,6.7 Inches,None,None,iPhone 15 Pro Max,...,None,Natural titanium,"€1,269.00","€1,479.00",€,4.6 out of 5 stars,https://www.amazon.nl/Apple-iPhone-Pro-Max-256...,https://m.media-amazon.com/images/I/81izs2I2Uv...,"[{'reviewer_name': 'Hamid', 'review_rating': '...",Amazon NL
3,Apple iPhone 15 Pro Max (256GB) - Black titanium,Apple,ios,8 GB,None,256 GB,6.7 Inches,None,None,iPhone 15 Pro Max,...,None,black titanium,"€1,259.00","€1,479.00",€,4.6 out of 5 stars,https://www.amazon.nl/Apple-iPhone-Pro-Max-256...,https://m.media-amazon.com/images/I/81fYvFvX-8...,"[{'reviewer_name': 'Hamid', 'review_rating': '...",Amazon NL
4,Apple iPhone 15 Pro Max (512GB) - Black titanium,Apple,ios,8 GB,None,512 GB,6.7 Inches,None,None,iPhone 15 Pro Max,...,None,black titanium,"€1,496.00","€1,729.00",€,4.6 out of 5 stars,https://www.amazon.nl/Apple-iPhone-Pro-Max-512...,https://m.media-amazon.com/images/I/81fYvFvX-8...,"[{'reviewer_name': 'Hamid', 'review_rating': '...",Amazon NL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Apple 2022 iPhone SE (256GB) - (PRODUCT)RED (3...,Apple,iOS 15,None,None,256 GB,4.7 Inches,None,None,iPhone,...,12 MP,(product)red,€566.95,€729.00,€,4.6 out of 5 stars,https://www.amazon.nl/Apple-2022-iPhone-256GB-...,https://m.media-amazon.com/images/I/61A0Zu4K-T...,"[{'reviewer_name': 'C K', 'review_rating': '5....",Amazon NL
92,Apple iPhone 13 (128GB) - Green,Apple,iOS 15,4 GB,None,128 GB,6.1 Inches,None,None,iPhone,...,None,green,€609.00,€749.00,€,4.6 out of 5 stars,https://www.amazon.nl/Apple-iPhone-13-128GB-Gr...,https://m.media-amazon.com/images/I/61-r9zOKBC...,"[{'reviewer_name': 'yakup', 'review_rating': '...",Amazon NL
93,Apple iPhone 13 (256GB) - Green,Apple,iOS 15,4 GB,None,256 GB,6.1 Inches,2532 x 1170,None,iPhone,...,None,Groen,€721.00,€879.00,€,4.6 out of 5 stars,https://www.amazon.nl/Apple-iPhone-13-256GB-Gr...,https://m.media-amazon.com/images/I/61-r9zOKBC...,"[{'reviewer_name': 'yakup', 'review_rating': '...",Amazon NL
94,Apple iPhone 13 (512GB) - Green,Apple,iOS 15,None,None,512 GB,6.1 Inches,2532 x 1170,None,iPhone,...,None,green,"€1,029.00","€1,129.00",€,4.6 out of 5 stars,https://www.amazon.nl/Apple-iPhone-13-512GB-Gr...,https://m.media-amazon.com/images/I/61-r9zOKBC...,"[{'reviewer_name': 'yakup', 'review_rating': '...",Amazon NL


In [ ]:
# import pandas as pd
# import re
# allcolors = [
#      "Rose Gold", "Navy Blue",
#     "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink",
#     "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black",
#     "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
#     "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White",
#     "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", 
#     "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", 
#     "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
#     "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", 
#     "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", 
#     "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
# ]
# import pandas as pd
# def find_color_in_title(title, allcolors):
#     for color in allcolors:
#         if color.lower() in title.lower():
#             return color
#     return None
# df_apple['Color'] = df_apple.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)
                                      #No need to do this step here

Amazon NL Samsung

In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import blinker
from fake_useragent import UserAgent

# Headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

driver = webdriver.Chrome()

urls = []
for page in range(1, 9):
    url = f'https://www.amazon.nl/-/en/s?i=electronics&rh=n%3A16366254031%2Cp_4%3ASAMSUNG%2Cp_n_condition-type%3A17170958031%2Cp_123%3A46655%2Cp_6%3AA17D2BRD4YMT0X&dc&page={page}&language=en_GB'

    driver.get(url)
    page_source = driver.page_source
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')))
    soup = BeautifulSoup(page_source, 'html.parser')
    url_tags = soup.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    for url_tag in url_tags:
        urls.append(f"https://www.amazon.nl{url_tag['href']}&language=en_GB")
driver.quit()
urls = list(set(urls))
print(f'{len(urls)} Product URLs retrieved.')

182 Product URLs retrieved.


In [33]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('div', id='cm-cr-global-review-list')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_pattern2 = re.compile(r'(\d+MP)\s+camera')
        camera_pattern3 = re.compile(r'(\d+ MP)\s+camera')
        camera_pattern4 = re.compile(r'(\d+ MP)\s+MAIN CAMERA')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            match2 = camera_pattern2.search(li.text)
            match3 = camera_pattern3.search(li.text)
            match4 = camera_pattern4.search(li.text)
            if match:
                camera_info = match.group(1)
                break
            if match2:
                camera_info = match2.group(1)
                break
            if match3:
                camera_info = match3.group(1)
                break
            if match4:
                camera_info = match4.group(1)
                break
        camera_features = None  # Placeholder for camera information
        
        # Find the table with camera features
        camera_features_row = soup.find('table', id='productDetails_techSpec_section_1')
        if camera_features_row:
            # Find the specific row containing "Other camera features"
            camera_features_row2 = camera_features_row.find('th', string='Other camera features')
            
            # Ensure the row exists and extract the camera details
            if camera_features_row2:
                camera_features = camera_features_row2.find_next('td').get_text(strip=True)
                
                # Extract the main camera's MP (assuming 'XXMP main camera' format)
                if 'main camera' in camera_features:
                    # Get the part before "main camera" and extract the megapixels
                    camera_info = camera_features.split('main camera')[0].strip().split()[-1]
        
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        color = None  # Initialize color
        
        if color_row:
            # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        
        elif soup.find('table', id ='productDetails_techSpec_section_1'):
            color_row2 = soup.find('th', string='Color')
            if color_row2:
                color = color_row2.find_next('td').get_text(strip=True)
        
        else:
            # Check for color in div with id='variation_color_name'
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = get_text_or_none(selection_row)
        price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                         if soup.find('div', class_='a-section a-spacing-micro') else None)
        if not price:
            price = get_text_or_none(soup.find('span', class_='a-price-whole'))



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': price,
            'Price Before Promotion' : get_text_or_none(soup.find('span', class_='a-price a-text-price').find('span', class_='a-offscreen') if soup.find('span', class_='a-price a-text-price') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('€')
            Website.append('Amazon NL')

# Create a DataFrame to store the scraped data
df_samsung = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_samsung

Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews sect

,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Model,...,Camera,Color,Price,Price Before Promotion,Currancy,Rate,URL,IMG,Reviews,Website
0,Sam Galaxy A20s EU-DS-32-3-4G-bk | Samsung com...,SAMSUNG,Android 9.0,None,None,3 GB,6.5 Inches,720 x 1560,None,Samsung A20s A207F (2020) Dual Sim 32GB Black EU,...,None,Black,€219.13,None,€,4.4 out of 5 stars,https://www.amazon.nl/-/en/Galaxy-EU-DS-32-3-4...,https://m.media-amazon.com/images/I/71VKC+67iM...,"[{'reviewer_name': 'AHAMADA AHMED', 'review_ra...",Amazon NL
1,Samsung Note20 LED View Cover,SAMSUNG,None,None,None,None,None,None,None,None,...,None,Copper Brown,€42.99,None,€,4.5 out of 5 stars,https://www.amazon.nl/-/en/Samsung-Note20-LED-...,https://m.media-amazon.com/images/I/81g2K1yHrH...,"[{'reviewer_name': 'Pinyin', 'review_rating': ...",Amazon NL
2,Samsung SM-A256 Galaxy A25 Dual SIM 5G 6GB RAM...,SAMSUNG,android,6 GB,None,128 GB,6.5 Inches,None,None,None,...,None,Black,€288.75,None,€,4.5 out of 5 stars,https://www.amazon.nl/-/en/Samsung-SM-A256-Gal...,https://m.media-amazon.com/images/I/61lqJH6WTq...,"[{'reviewer_name': 'Sebastiaan Vermeij', 'revi...",Amazon NL
3,"Samsung Galaxy S22, Mobile Phone, 5G, 128GB, L...",SAMSUNG,Android 12.0,None,None,8 GB,6.1 Inches,None,None,"S22 Lavender, 128 GB",...,None,lavender,€892.63,None,€,4.2 out of 5 stars,https://www.amazon.nl/-/en/Samsung-Lavender-In...,https://m.media-amazon.com/images/I/71izi4F1WN...,"[{'reviewer_name': 'Anna', 'review_rating': 'N...",Amazon NL
4,Samsung compatible Galaxy S22 Ultra 5G Black 1...,SAMSUNG,android,None,None,8 GB,6.8 Inches,None,None,None,...,None,Black,"€1,286.44",None,€,4.3 out of 5 stars,https://www.amazon.nl/-/en/Samsung-compatible-...,https://m.media-amazon.com/images/I/81-Zt1Q15F...,"[{'reviewer_name': 'jacqueline bailly', 'revie...",Amazon NL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,Galaxy Z Flip3 5G 256GB Version EU,SAMSUNG,android,None,None,8 GB,6.7 Inches,None,None,Galaxy Z Flip 3,...,None,Black,"€1,029.39",None,€,3.8 out of 5 stars,https://www.amazon.nl/-/en/Galaxy-Flip3-5G-256...,https://m.media-amazon.com/images/I/517vr7sbJB...,"[{'reviewer_name': 'manissier fabrice', 'revie...",Amazon NL
178,Samsung GSM005445 pencil case,SAMSUNG,None,None,None,None,1.6,None,None,None,...,None,Transparant,€9.10,None,€,5.0 out of 5 stars,https://www.amazon.nl/-/en/GSM005445-Samsung-p...,https://m.media-amazon.com/images/I/31xswovix0...,[],Amazon NL
179,Samsung Galaxy Book3 Pro - 14 inch i7-1360P 16...,SAMSUNG,android,16 GB,None,None,14 Inches,None,None,Galaxy Book3 Pro,...,None,Black,"€2,140.63",None,€,Previous page,https://www.amazon.nl/-/en/Samsung-Galaxy-Book...,https://m.media-amazon.com/images/I/61wu1GOX+2...,[],Amazon NL
180,Samsung Galaxy A25 5G Blue 128gb 6gb,SAMSUNG,Android 14,None,None,128 GB,6.44 Inches,None,None,Galaxy A25,...,None,Black,€199.00,None,€,4.4 out of 5 stars,https://www.amazon.nl/-/en/Samsung-Galaxy-A25-...,https://m.media-amazon.com/images/I/41T+r5k0d1...,"[{'reviewer_name': 'Paulina', 'review_rating':...",Amazon NL


In [ ]:
# import pandas as pd
# import re
# allcolors = [
#      "Rose Gold", "Navy Blue",
#     "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink",
#     "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black",
#     "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
#     "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White",
#     "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", 
#     "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", 
#     "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
#     "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", 
#     "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", 
#     "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
# ]
# import pandas as pd
# def find_color_in_title(title, allcolors):
#     for color in allcolors:
#         if color.lower() in title.lower():
#             return color
#     return None
# df_samsung['Color'] = df_samsung.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

Amazon NL XIAOMI

In [36]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import blinker
from fake_useragent import UserAgent

# Headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

driver = webdriver.Chrome()

urls = []
for page in range(1, 7):
    url = f'https://www.amazon.nl/s?i=electronics&rh=n%3A16366059031%2Cp_4%3AXiaomi%2Cp_6%3AA17D2BRD4YMT0X&dc&page={page}&language=en_GB'

    driver.get(url)
    page_source = driver.page_source
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')))
    soup = BeautifulSoup(page_source, 'html.parser')
    url_tags = soup.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    for url_tag in url_tags:
        urls.append(f"https://www.amazon.nl{url_tag['href']}&language=en_GB")
driver.quit()
urls = list(set(urls))
print(f'{len(urls)} Product URLs retrieved.')

141 Product URLs retrieved.


In [37]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('span', class_='global-reviews-all')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_pattern2 = re.compile(r'(\d+MP)\s+camera')
        camera_pattern3 = re.compile(r'(\d+ MP)\s+camera')
        camera_pattern4 = re.compile(r'(\d+ MP)\s+MAIN CAMERA')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            match2 = camera_pattern2.search(li.text)
            match3 = camera_pattern3.search(li.text)
            match4 = camera_pattern4.search(li.text)
            if match:
                camera_info = match.group(1)
                break
            if match2:
                camera_info = match2.group(1)
                break
            if match3:
                camera_info = match3.group(1)
                break
            if match4:
                camera_info = match4.group(1)
                break
        camera_features = None  # Placeholder for camera information
        
        # Find the table with camera features
        camera_features_row = soup.find('table', id='productDetails_techSpec_section_1')
        if camera_features_row:
            # Find the specific row containing "Other camera features"
            camera_features_row2 = camera_features_row.find('th', string='Other camera features')
            
            # Ensure the row exists and extract the camera details
            if camera_features_row2:
                camera_features = camera_features_row2.find_next('td').get_text(strip=True)
                
                # Extract the main camera's MP (assuming 'XXMP main camera' format)
                if 'main camera' in camera_features:
                    # Get the part before "main camera" and extract the megapixels
                    camera_info = camera_features.split('main camera')[0].strip().split()[-1]
        
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        color = None  # Initialize color
        
        if color_row:
            # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        
        elif soup.find('table', id ='productDetails_techSpec_section_1'):
            color_row2 = soup.find('th', string='Color')
            if color_row2:
                color = color_row2.find_next('td').get_text(strip=True)
        
        else:
            # Check for color in div with id='variation_color_name'
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = get_text_or_none(selection_row)
        price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                         if soup.find('div', class_='a-section a-spacing-micro') else None)
        if not price:
            price = get_text_or_none(soup.find('span', class_='a-price-whole'))



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': price,
            'Price Before Promotion' : get_text_or_none(soup.find('span', class_='a-price a-text-price').find('span', class_='a-offscreen') if soup.find('span', class_='a-price a-text-price') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('€')
            Website.append('Amazon NL')

# Create a DataFrame to store the scraped data
df_xiaomi = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_xiaomi

Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found


,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Model,...,Camera,Color,Price,Price Before Promotion,Currancy,Rate,URL,IMG,Reviews,Website
0,"Xiaomi Mi 11 Lite 8+128GB 6.55"" 5G Citrus Yell...",Xiaomi,Android 11.0,None,None,8 GB,6.55 Inches,None,None,Xiaomi Mi 11 Lite 5G,...,None,Yellow,€224.09,None,€,5.0 out of 5 stars,https://www.amazon.nl/-/en/Xiaomi-Lite-128GB-C...,https://m.media-amazon.com/images/I/61jkurfttz...,[],Amazon NL
1,Xiaomi Redmi Note 13 Pro+ 5G 2.8GHz 12GB+512GB...,None,None,None,None,None,None,None,None,None,...,None,None,€468.90,None,€,5.0 out of 5 stars,https://www.amazon.nl/-/en/Xiaomi-Redmi-2-8GHz...,https://m.media-amazon.com/images/I/511DMgIAum...,[],Amazon NL
2,"Xiaomi Redmi Note 5 (Hybrid Dual SIM) -P, DE/F...",Xiaomi,android,3 GB,None,32 GB,5.99 Inches,1920 x 1080,None,Redmi Note 5,...,None,None,€200.55,None,€,4.4 out of 5 stars,https://www.amazon.nl/-/en/Xiaomi-Redmi-Note-H...,https://m.media-amazon.com/images/I/51G009hFg-...,"[{'reviewer_name': 'Jorge Flores', 'review_rat...",Amazon NL
3,"Xiaomi Redmi 10 (2022) - Smartphone 64GB, 4GB ...",Xiaomi,Android 11.0,None,None,4 GB,6.5 Inches,None,2.05 GHz,Xiaomi Redmi 10 (2022),...,None,Pebble White,€240.44,None,€,5.0 out of 5 stars,https://www.amazon.nl/-/en/Xiaomi-Redmi-10-Sma...,https://m.media-amazon.com/images/I/61FPEsbM8P...,[],Amazon NL
4,"Xiaomi 2201122G 12 Pro Smartphone, 16.73 cm (6...",Xiaomi,MIUI 12,None,Snapdragon,256 GB,6.73 Inches,None,None,Xiaomi 12 Pro,...,None,None,€887.95,None,€,3.8 out of 5 stars,https://www.amazon.nl/-/en/Xiaomi-2201122G-Sma...,https://m.media-amazon.com/images/I/51K-P9Dnv8...,"[{'reviewer_name': 'Gianluca Ferente', 'review...",Amazon NL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Xiaomi 12 Purple 8GB RAM 128GB ROM,Xiaomi,android,8 GB,None,128 GB,6.28 Inches,1080 x 2400,3 GHz,Xiaomi 12 UK,...,None,None,€382.19,None,€,4.3 out of 5 stars,https://www.amazon.nl/-/en/Xiaomi-Purple-8GB-R...,https://m.media-amazon.com/images/I/413W+dTx27...,"[{'reviewer_name': 'Fermin', 'review_rating': ...",Amazon NL
137,"Xiaomi 11 Lite 5G NE Smartphone 128 GB, 8 GB R...",Xiaomi,MIUI 12,8 GB,None,128 GB,6.55 Inches,None,2.4 GHz,Xiaomi 11 Lite 5G NE,...,None,None,€373.14,None,€,4.3 out of 5 stars,https://www.amazon.nl/-/en/Xiaomi-Lite-Smartph...,https://m.media-amazon.com/images/I/51vy5iENUc...,"[{'reviewer_name': 'Alexander Schechinger', 'r...",Amazon NL
138,"Xiaomi 12 Pro Smartphone, 6.73 inches, AMOLED,...",Xiaomi,Android 12.0,12 GB,Snapdragon,256 GB,None,None,"3.0, 2.5, 1.8 GHz",Xiaomi 12 Pro 5G,...,None,None,"€1,005.77",None,€,4.2 out of 5 stars,https://www.amazon.nl/-/en/Smartphone-Unlocked...,https://m.media-amazon.com/images/I/616bJyY3g3...,"[{'reviewer_name': 'stefania zigiotto', 'revie...",Amazon NL
139,"Xiaomi Smartphone 14,Snapdragon 8 Gen3 3.3GHz,...",Xiaomi,MIUI 14,512 GB,None,512 GB,6.36 Inches,2670 x 1200,3.3 GHz,Xiaomi 14,...,None,None,€789.90,None,€,4.2 out of 5 stars,https://www.amazon.nl/-/en/6941812783092/dp/B0...,https://m.media-amazon.com/images/I/51LAXd-S1-...,"[{'reviewer_name': 'chispolas', 'review_rating...",Amazon NL


In [38]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue",
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White",
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", 
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", 
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", 
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", 
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
import pandas as pd
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_xiaomi['Color'] = df_xiaomi.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

Amazon NL OPPO

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import blinker
import time
from fake_useragent import UserAgent

# Headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

driver = webdriver.Firefox()

urls = []
for page in range(1, 14):
    url = f'https://www.amazon.nl/s?i=electronics&rh=n%3A16366059031%2Cp_4%3AOppo&page={page}&language=en_GB'

    driver.get(url)
    page_source = driver.page_source
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')))
    time.sleep(2)
    soup = BeautifulSoup(page_source, 'html.parser')
    url_tags = soup.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    for url_tag in url_tags:
        urls.append(f"https://www.amazon.nl{url_tag['href']}&language=en_GB")
driver.quit()
urls = list(set(urls))
print(f'{len(urls)} Product URLs retrieved.')

303 Product URLs retrieved.


In [8]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('span', class_='global-reviews-all')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_pattern2 = re.compile(r'(\d+MP)\s+camera')
        camera_pattern3 = re.compile(r'(\d+ MP)\s+camera')
        camera_pattern4 = re.compile(r'(\d+ MP)\s+MAIN CAMERA')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            match2 = camera_pattern2.search(li.text)
            match3 = camera_pattern3.search(li.text)
            match4 = camera_pattern4.search(li.text)
            if match:
                camera_info = match.group(1)
                break
            if match2:
                camera_info = match2.group(1)
                break
            if match3:
                camera_info = match3.group(1)
                break
            if match4:
                camera_info = match4.group(1)
                break
        camera_features = None  # Placeholder for camera information
        
        # Find the table with camera features
        camera_features_row = soup.find('table', id='productDetails_techSpec_section_1')
        if camera_features_row:
            # Find the specific row containing "Other camera features"
            camera_features_row2 = camera_features_row.find('th', string='Other camera features')
            
            # Ensure the row exists and extract the camera details
            if camera_features_row2:
                camera_features = camera_features_row2.find_next('td').get_text(strip=True)
                
                # Extract the main camera's MP (assuming 'XXMP main camera' format)
                if 'main camera' in camera_features:
                    # Get the part before "main camera" and extract the megapixels
                    camera_info = camera_features.split('main camera')[0].strip().split()[-1]
        
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        color = None  # Initialize color
        
        if color_row:
            # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        
        elif soup.find('table', id ='productDetails_techSpec_section_1'):
            color_row2 = soup.find('th', string='Color')
            if color_row2:
                color = color_row2.find_next('td').get_text(strip=True)
        
        else:
            # Check for color in div with id='variation_color_name'
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = get_text_or_none(selection_row)
        price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                         if soup.find('div', class_='a-section a-spacing-micro') else None)
        if not price:
            price = get_text_or_none(soup.find('span', class_='a-price-whole'))



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': price,
            'Price Before Promotion' : get_text_or_none(soup.find('span', class_='a-price a-text-price').find('span', class_='a-offscreen') if soup.find('span', class_='a-price a-text-price') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('€')
            Website.append('Amazon NL')

# Create a DataFrame to store the scraped data
df_oppo = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_oppo

Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews sect

,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Model,...,Camera,Color,Price,Price Before Promotion,Currancy,Rate,URL,IMG,Reviews,Website
0,"OPPO Reno 7 Mobile Phone Free, 8GB + 128GB, Ca...",OPPO,"ColorOs 12.1, Android 12.0",8 GB,None,128 GB,6.43 Inches,1080 x 2400 pixels,None,Reindeer 7,...,None,None,€321.65,None,€,4.3 out of 5 stars,https://www.amazon.nl/-/en/Reno-Android-Smartp...,https://m.media-amazon.com/images/I/61o0uQbM1L...,"[{'reviewer_name': 'Carlos', 'review_rating': ...",Amazon NL
1,"OPPO A16-64GB - Crystal Black,Black",OPPO,Android 10.0,None,None,4 GB,6.52 Inches,None,None,A16,...,None,Black,145.,€145.00,€,4.3 out of 5 stars,https://www.amazon.nl/-/en/OPPO-A16-64GB-Cryst...,https://m.media-amazon.com/images/I/71lPDHSgCw...,"[{'reviewer_name': 'LRU1969', 'review_rating':...",Amazon NL
2,OPPO A57s Smartphone - 16.66 cm (6.56 inch) LC...,OPPO,"ColorOS 12.1, Android 12.0",4 GB,MediaTek Helio,4 GB,6.56 Inches,None,None,A57s,...,None,None,166.,€166.95,€,3.8 out of 5 stars,https://www.amazon.nl/-/en/OPPO-A57s-Smartphon...,https://m.media-amazon.com/images/I/51PywB+y-D...,"[{'reviewer_name': 'andrea', 'review_rating': ...",Amazon NL
3,OPPO Reno 8 Pro Smartphone 5G - 8GB RAM + 256G...,OPPO,"android, ColorOS 12 based on Android 11",8 GB,None,8 GB,6.7 Inches,None,None,Model Specific,...,None,Ice green,649.,€649.28,€,4.1 out of 5 stars,https://www.amazon.nl/-/en/RENO-8-PRO-VERT/dp/...,https://m.media-amazon.com/images/I/71ZntbH2+9...,"[{'reviewer_name': 'Cristian', 'review_rating'...",Amazon NL
4,"OPPO RENO8 Smartphone 5G, AI Triple Camera 50 ...",OPPO,Android 12.0,8 GB,None,256 GB,6.43 Inches,2400 x 1080 pixels,None,Reno8,...,None,None,195.,None,€,4.3 out of 5 stars,https://www.amazon.nl/-/en/Smartphone-Triple-D...,https://m.media-amazon.com/images/I/61f6+aAp1V...,"[{'reviewer_name': 'Annalisa Panaro', 'review_...",Amazon NL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,"Oppo Find X5 Pro 5G - Smartphone 256GB, 12GB R...",OPPO,Android 12.0,None,None,12 GB,6.7 Inches,None,None,Oppo Find X5 Pro 5G,...,None,Black,386.,None,€,3.2 out of 5 stars,https://www.amazon.nl/-/en/Oppo-Find-X5-Pro-Sm...,https://m.media-amazon.com/images/I/71wvEt9ODw...,"[{'reviewer_name': 'Erika', 'review_rating': '...",Amazon NL
299,OPPO A80 5G,OPPO,Android 14,8 GB,None,256 GB,6.67 Inches,None,None,OPPO A80 5G,...,None,None,€307.80,None,€,Previous page of related Sponsored Products,https://www.amazon.nl/-/en/CPH2639SK-OPPO-A80-...,https://m.media-amazon.com/images/I/51O1MIzu3o...,[],Amazon NL
300,"OPPO A16s Smartphone 64GB, 4GB RAM, Dual-SIM, ...",OPPO,"android, ColorOS 11.1,Andorid 11.0",4 GB,None,64 GB,6.52 Inches,None,None,A16S,...,None,None,113.,None,€,Previous page of related Sponsored Products,https://www.amazon.nl/-/en/OPPO-A16s-Smartphon...,https://m.media-amazon.com/images/I/61NE0Lk5bW...,[],Amazon NL
301,"OPPO A17 DS 4GB/64GB, Blue EU",OPPO,None,None,None,64 GB,None,None,None,None,...,None,None,€164.99,None,€,5.0 out of 5 stars,https://www.amazon.nl/-/en/OPPO-A17-4GB-64GB-B...,https://m.media-amazon.com/images/I/71NdpWKz6F...,[],Amazon NL


In [9]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue",
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White",
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", 
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", 
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", 
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", 
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
import pandas as pd
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_oppo['Color'] = df_oppo.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

Amazon NL NOKIA

In [12]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time
from bs4 import BeautifulSoup
import pandas as pd



urls = []
driver = webdriver.Chrome()
mainurl = 'https://www.amazon.nl/stores/page/B8D1C27D-1143-4F75-BFCB-80F206421460?ingress=2&visitId=96422b05-5782-4a71-93ef-33ccd4483f5f&ref_=ast_bln&language=en_GB'
driver.get(mainurl)

while True:
    try:
        WebDriverWait(driver, 20).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, 'loading-mask'))
        )
        
        # Update the button locator to something more reliable (using XPath as an example)
        show_more_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ShowMoreButton")]'))
        )
        
        time.sleep(5)  # Use a smaller sleep and test the timing.
        show_more_button.click()
        
        # Wait for the product grid items to load
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'a.Overlay__overlay__LloCU.ProductGridItem__overlay__IQ3Kw'))
        )
    except TimeoutException:
        print("End of content or timeout reached.")
        break
    except Exception as e:
        print(f"Error: {e}")
        break

# Parse the page content after clicking 'Show More'
soup = BeautifulSoup(driver.page_source, 'html.parser')
url_tags = soup.find_all('a', class_='Overlay__overlay__LloCU')
for url_tag in url_tags:
    urls.append(f"https://www.amazon.nl{url_tag['href']}&language=en_GB")

driver.quit()
print(f'{len(urls)} URLs Retrieved')

End of content or timeout reached.
15 URLs Retrieved


In [13]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('span', class_='global-reviews-all')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_pattern2 = re.compile(r'(\d+MP)\s+camera')
        camera_pattern3 = re.compile(r'(\d+ MP)\s+camera')
        camera_pattern4 = re.compile(r'(\d+ MP)\s+MAIN CAMERA')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            match2 = camera_pattern2.search(li.text)
            match3 = camera_pattern3.search(li.text)
            match4 = camera_pattern4.search(li.text)
            if match:
                camera_info = match.group(1)
                break
            if match2:
                camera_info = match2.group(1)
                break
            if match3:
                camera_info = match3.group(1)
                break
            if match4:
                camera_info = match4.group(1)
                break
        camera_features = None  # Placeholder for camera information
        
        # Find the table with camera features
        camera_features_row = soup.find('table', id='productDetails_techSpec_section_1')
        if camera_features_row:
            # Find the specific row containing "Other camera features"
            camera_features_row2 = camera_features_row.find('th', string='Other camera features')
            
            # Ensure the row exists and extract the camera details
            if camera_features_row2:
                camera_features = camera_features_row2.find_next('td').get_text(strip=True)
                
                # Extract the main camera's MP (assuming 'XXMP main camera' format)
                if 'main camera' in camera_features:
                    # Get the part before "main camera" and extract the megapixels
                    camera_info = camera_features.split('main camera')[0].strip().split()[-1]
        
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        color = None  # Initialize color
        
        if color_row:
            # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        
        elif soup.find('table', id ='productDetails_techSpec_section_1'):
            color_row2 = soup.find('th', string='Color')
            if color_row2:
                color = color_row2.find_next('td').get_text(strip=True)
        
        else:
            # Check for color in div with id='variation_color_name'
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = get_text_or_none(selection_row)
        price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                         if soup.find('div', class_='a-section a-spacing-micro') else None)
        if not price:
            price = get_text_or_none(soup.find('span', class_='a-price-whole'))



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': price,
            'Price Before Promotion' : get_text_or_none(soup.find('span', class_='a-price a-text-price').find('span', class_='a-offscreen') if soup.find('span', class_='a-price a-text-price') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('€')
            Website.append('Amazon NL')

# Create a DataFrame to store the scraped data
df_nokia = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    # 'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_nokia

An error occurred while processing https://www.amazon.nl/stores/page/61C68484-8A24-4E61-B6EB-57E9EE6496FF?ingress=2&visitId=3e40b23a-b16b-4cdc-a4a9-0002cf63fcfd&ref_=ast_bln&language=en_GB: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16
An error occurred while processing https://www.amazon.nl/stores/page/73B5E312-D6B3-4310-9ADA-A23B0B099941?ingress=2&visitId=3e40b23a-b16b-4cdc-a4a9-0002cf63fcfd&ref_=ast_bln&language=en_GB: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


ValueError: All arrays must be of the same length

In [15]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue",
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White",
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", 
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", 
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", 
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", 
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
import pandas as pd
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_nokia['Color'] = df_nokia.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

Amazon NL HUAWEI

In [19]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
import time
from bs4 import BeautifulSoup
import pandas as pd


urls = []
driver = webdriver.Chrome()
mainurl = 'https://www.amazon.nl/stores/page/76B158C0-EA46-4C68-993F-B28C8CBEE29E?ingress=2&visitId=1cdeee24-ece7-4174-af95-e6e36deaed95&ref_=ast_bln&language=en_GB'
driver.get(mainurl)

while True:
    try:
        WebDriverWait(driver, 20).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, 'loading-mask'))
        )
        
        # Update the button locator to something more reliable (using XPath as an example)
        show_more_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ShowMoreButton")]'))
        )
        
        time.sleep(5)  # Use a smaller sleep and test the timing.
        show_more_button.click()
        
        # Wait for the product grid items to load
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'a.Overlay__overlay__LloCU.ProductGridItem__overlay__IQ3Kw'))
        )
    except TimeoutException:
        print("End of content or timeout reached.")
        break
    except Exception as e:
        print(f"Error: {e}")
        break

# Parse the page content after clicking 'Show More'
soup = BeautifulSoup(driver.page_source, 'html.parser')
url_tags = soup.find_all('a', class_='Overlay__overlay__LloCU')
for url_tag in url_tags:
    urls.append(f"https://www.amazon.nl{url_tag['href']}&language=en_GB")

driver.quit()
print(f'{len(urls)} URLs Retrieved')

End of content or timeout reached.
16 URLs Retrieved


In [20]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('span', class_='global-reviews-all')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_pattern2 = re.compile(r'(\d+MP)\s+camera')
        camera_pattern3 = re.compile(r'(\d+ MP)\s+camera')
        camera_pattern4 = re.compile(r'(\d+ MP)\s+MAIN CAMERA')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            match2 = camera_pattern2.search(li.text)
            match3 = camera_pattern3.search(li.text)
            match4 = camera_pattern4.search(li.text)
            if match:
                camera_info = match.group(1)
                break
            if match2:
                camera_info = match2.group(1)
                break
            if match3:
                camera_info = match3.group(1)
                break
            if match4:
                camera_info = match4.group(1)
                break
        camera_features = None  # Placeholder for camera information
        
        # Find the table with camera features
        camera_features_row = soup.find('table', id='productDetails_techSpec_section_1')
        if camera_features_row:
            # Find the specific row containing "Other camera features"
            camera_features_row2 = camera_features_row.find('th', string='Other camera features')
            
            # Ensure the row exists and extract the camera details
            if camera_features_row2:
                camera_features = camera_features_row2.find_next('td').get_text(strip=True)
                
                # Extract the main camera's MP (assuming 'XXMP main camera' format)
                if 'main camera' in camera_features:
                    # Get the part before "main camera" and extract the megapixels
                    camera_info = camera_features.split('main camera')[0].strip().split()[-1]
        
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        color = None  # Initialize color
        
        if color_row:
            # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        
        elif soup.find('table', id ='productDetails_techSpec_section_1'):
            color_row2 = soup.find('th', string='Color')
            if color_row2:
                color = color_row2.find_next('td').get_text(strip=True)
        
        else:
            # Check for color in div with id='variation_color_name'
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = get_text_or_none(selection_row)
        price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                         if soup.find('div', class_='a-section a-spacing-micro') else None)
        if not price:
            price = get_text_or_none(soup.find('span', class_='a-price-whole'))



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': price,
            'Price Before Promotion' : get_text_or_none(soup.find('span', class_='a-price a-text-price').find('span', class_='a-offscreen') if soup.find('span', class_='a-price a-text-price') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('€')
            Website.append('Amazon NL')

# Create a DataFrame to store the scraped data
df_huawei = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    # 'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_huawei

An error occurred while processing https://www.amazon.nl/stores/page/F4188C82-61CC-42B5-B474-B871C4C190E5?ingress=2&visitId=e5ae59e1-3d25-4769-95de-ab6c69aefa2c&ref_=ast_bln&language=en_GB: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

An error occurred while processing https://www.amazon.nl/stores/page/7CFCA80C-80E6-4543-8B37-3718C249BE52?ingress=2&visitId=e5ae59e1-3d25-4769-95de-ab6c69aefa2c&ref_=ast_bln&language=en_GB: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Model,...,Cellular Technology,Camera,Color,Price,Price Before Promotion,Currancy,Rate,IMG,Reviews,Website
0,"HUAWEI Pura 70 Ultra Smartphone, 16GB+ 512GB, ...",HUAWEI,EMUI 14.2,None,None,16 GB,1 Inches,None,None,HUAWEI Pura 70 Ultra Green 16GB+512GB,...,4G,None,green,"1,483.","€1,483.82",€,3.8 out of 5 stars,https://m.media-amazon.com/images/I/71-W9YVyc5...,"[{'reviewer_name': 'Serena', 'review_rating': ...",Amazon NL
1,"HUAWEI Pura 70 Ultra Smartphone, 16GB+ 512GB, ...",HUAWEI,EMUI 14.2,None,None,16 GB,1 Inches,None,None,HUAWEI Pura 70 Ultra Green 16GB+512GB,...,4G,None,green,"1,483.","€1,483.82",€,3.8 out of 5 stars,https://m.media-amazon.com/images/I/71-W9YVyc5...,"[{'reviewer_name': 'Serena', 'review_rating': ...",Amazon NL
2,"HUAWEI Pura 70 Pro Smartphone, 12GB RAM + 512G...",HUAWEI,EMUI 14.2,None,None,12 GB,6.4,None,None,HUAWEI Pura 70 Pro White 12GB+512GB,...,4G,None,White,"€1,069.71",None,€,3.8 out of 5 stars,https://m.media-amazon.com/images/I/61FuXcC5EL...,"[{'reviewer_name': 'Serena', 'review_rating': ...",Amazon NL
3,"HUAWEI Pura 70 Smartphone, 12GB + 256GB, Flat ...",HUAWEI,EMUI 14.2,None,None,12 GB,6.4,None,None,Audrey-L29CK,...,4G,None,Black,980.,€980.50,€,3.8 out of 5 stars,https://m.media-amazon.com/images/I/71z-aobAO8...,"[{'reviewer_name': 'Serena', 'review_rating': ...",Amazon NL
4,"HUAWEI P60 Pro, Ultra Lighting XMAGE Camera, 8...",HUAWEI,EMUI 13.1,8 GB,Snapdragon,8 GB,6.67 Inches,None,None,P60 Pro,...,4G,None,None,"1,483.",None,€,3.9 out of 5 stars,"data:image/jpeg;base64,R0lGODlhAQABAIAAAAAAAP/...","[{'reviewer_name': 'Toni P.', 'review_rating':...",Amazon NL
5,"HUAWEI P60 Pro, Ultra Lighting XMAGE Camera, 8...",HUAWEI,EMUI 13.1,8 GB,Snapdragon,8 GB,6.67 Inches,None,None,P60 Pro,...,4G,None,None,"1,199.","€1,199.00",€,3.5 out of 5 stars,https://m.media-amazon.com/images/I/5164NoUSjg...,"[{'reviewer_name': 'Drachenritter', 'review_ra...",Amazon NL
6,"HUAWEI nova 11i, 8GB + 128GB, 6.8"" FullView Di...",HUAWEI,EMUI 13,8 GB,None,8 GB,6.8 Inches,None,None,Nova 11i,...,4G,None,Yellow,252.,None,€,3.8 out of 5 stars,https://m.media-amazon.com/images/I/51ZOKIbcuZ...,"[{'reviewer_name': 'M', 'review_rating': 'N/A'...",Amazon NL
7,"HUAWEI nova 12i Smart Phone, 6.7-inch AOD Disp...",HUAWEI,EMUI 14,8 GB,None,8 GB,6.7 Inches,None,None,Cartier-L81F,...,4G,None,green,259.,€279.00,€,3.7 out of 5 stars,https://m.media-amazon.com/images/I/81fY0h++NW...,"[{'reviewer_name': 'Iriskhan', 'review_rating'...",Amazon NL
8,HUAWEI Mate X3 Slim Lightweight Foldable Quad-...,HUAWEI,EMUI 13.1,12 GB,Snapdragon,12 GB,7.85 Inches,None,None,Mate X3,...,4G,None,None,6.,None,€,3.8 out of 5 stars,https://m.media-amazon.com/images/I/51cjy7IGFW...,"[{'reviewer_name': 'MiMaestro', 'review_rating...",Amazon NL
9,"HUAWEI P60 Pro, Ultra Lighting XMAGE Camera, 8...",HUAWEI,EMUI 13.1,8 GB,Snapdragon,8 GB,6.67 Inches,None,None,P60 Pro,...,4G,None,None,"1,199.","€1,199.00",€,3.5 out of 5 stars,https://m.media-amazon.com/images/I/5164NoUSjg...,"[{'reviewer_name': 'Drachenritter', 'review_ra...",Amazon NL


In [21]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue",
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White",
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", 
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", 
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", 
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", 
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
import pandas as pd
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_huawei['Color'] = df_huawei.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

Final Result (Before Cleaning)

In [ ]:
df = pd.concat([df_nokia, df_samsung, df_apple, df_xiaomi, df_oppo, df_huawei], ignore_index=True)

In [ ]:
df.to_csv('Final Amazon NL.csv', index = False)